In [1]:
from tensorflow.keras.datasets import cifar10

import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, Lambda, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.applications.vgg19 import decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model
import cv2
from keras_preprocessing.image import ImageDataGenerator
import os
import scipy.io
import tensorflow as tf
import json
import sys
sys.path.append('..')  
from GradPri_utils.utils import *

from tensorflow.keras.losses import categorical_crossentropy
from progressbar import ProgressBar
import os

In [2]:
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()
print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
num_classes = 10
# Convert class vectors to binary class matrices.
Y_train = to_categorical(Y_train, num_classes)
Y_test = to_categorical(Y_test, num_classes)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [11]:
input_shape = (32,32,3)

model = Sequential()
# model.add(Lambda(augment_2d,
#                  input_shape=x_train.shape[1:],
#                  arguments={'rotation': 8.0, 'horizontal_flip': True}))
# Block 1
model.add(Conv2D(64, (3, 3), padding='same', name = 'block1_conv1', input_shape=input_shape))
model.add(BatchNormalization(name = 'block1_bn1'))
model.add(Activation('relu', name = 'block1_act1'))

model.add(Conv2D(64, (3, 3), padding='same', name = 'block1_conv2'))
model.add(BatchNormalization(name = 'block1_bn2'))
model.add(Activation('relu', name = 'block1_act2'))

model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='block1_pool'))
model.add(Dropout(0.25, name='block1_drop'))

# Block 2
model.add(Conv2D(128, (3, 3), padding='same', name = 'block2_conv1'))
model.add(BatchNormalization(name = 'block2_bn1'))
model.add(Activation('relu', name = 'block2_act1'))

model.add(Conv2D(128, (3, 3), padding='same', name = 'block2_conv2'))
model.add(BatchNormalization(name = 'block2_bn2'))
model.add(Activation('relu', name = 'block2_act2'))

model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='block2_pool'))
model.add(Dropout(0.25, name='block2_drop'))

# Block 3
model.add(Conv2D(256, (3, 3), padding='same', name = 'block3_conv1'))
model.add(BatchNormalization(name = 'block3_bn1'))
model.add(Activation('relu', name = 'block3_act1'))

model.add(Conv2D(256, (3, 3), padding='same', name = 'block3_conv2'))
model.add(BatchNormalization(name = 'block3_bn2'))
model.add(Activation('relu', name = 'block3_act2'))

model.add(Conv2D(256, (3, 3), padding='same', name = 'block3_conv3'))
model.add(BatchNormalization(name = 'block3_bn3'))
model.add(Activation('relu', name = 'block3_act3'))

model.add(Conv2D(256, (3, 3), padding='same', name = 'block3_conv4'))
model.add(BatchNormalization(name = 'block3_bn4'))
model.add(Activation('relu', name = 'block3_act4'))

model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='block3_pool'))
model.add(Dropout(0.25, name='block3_drop'))

# Block 4
model.add(Conv2D(512, (3, 3), padding='same', name = 'block4_conv1'))
model.add(BatchNormalization(name = 'block4_bn1'))
model.add(Activation('relu', name = 'block4_act1'))

model.add(Conv2D(512, (3, 3), padding='same', name = 'block4_conv2'))
model.add(BatchNormalization(name = 'block4_bn2'))
model.add(Activation('relu', name = 'block4_act2'))

model.add(Conv2D(512, (3, 3), padding='same', name = 'block4_conv3'))
model.add(BatchNormalization(name = 'block4_bn3'))
model.add(Activation('relu', name = 'block4_act3'))

model.add(Conv2D(512, (3, 3), padding='same', name = 'block4_conv4'))
model.add(BatchNormalization(name = 'block4_bn4'))
model.add(Activation('relu', name = 'block4_act4'))

model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='block4_pool'))
model.add(Dropout(0.25, name='block4_drop'))

# Block 5
model.add(Conv2D(512, (3, 3), padding='same', name = 'block5_conv1'))
model.add(BatchNormalization(name = 'block5_bn1'))
model.add(Activation('relu', name = 'block5_act1'))

model.add(Conv2D(512, (3, 3), padding='same', name = 'block5_conv2'))
model.add(BatchNormalization(name = 'block5_bn2'))
model.add(Activation('relu', name = 'block5_act2'))

model.add(Conv2D(512, (3, 3), padding='same', name = 'block5_conv3'))
model.add(BatchNormalization(name = 'block5_bn3'))
model.add(Activation('relu', name = 'block5_act3'))

model.add(Conv2D(512, (3, 3), padding='same', name = 'block5_conv4'))
model.add(BatchNormalization(name = 'block5_bn4'))
model.add(Activation('relu', name = 'block5_act4'))

model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='block5_pool'))
model.add(Dropout(0.25, name='block5_drop'))

# include_top
model.add(Flatten(name='flatten'))
model.add(Dense(4096, activation='relu', name='fc1'))
model.add(Dropout(0.5, name='drop1'))
model.add(Dense(4096, activation='relu', name='fc2'))
model.add(Dropout(0.5, name='drop2'))
model.add(Dense(num_classes, activation='softmax', name='predictions'))


In [12]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_bn1 (BatchNormalizati (None, 32, 32, 64)        256       
_________________________________________________________________
block1_act1 (Activation)     (None, 32, 32, 64)        0         
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_bn2 (BatchNormalizati (None, 32, 32, 64)        256       
_________________________________________________________________
block1_act2 (Activation)     (None, 32, 32, 64)        0         
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)       

In [22]:
model = load_model("./cifar10_VGG19.h5")
# # initiate RMSprop optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.00005)

# # Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath="./cifar10_VGG19.h5", 
                                                monitor='val_accuracy', 
                                                verbose=1, save_best_only=True, mode = 'max')
callbacks_list = [checkpoint]

model.fit(X_train, Y_train, batch_size=256, epochs=20, shuffle=True,
          validation_data=(X_test[0:5000], Y_test[0:5000]), callbacks=callbacks_list)

Epoch 1/20
196/196 [==============================] - 9s 35ms/step - loss: 0.3634 - accuracy: 0.8797 - val_loss: 0.4848 - val_accuracy: 0.8468

Epoch 00001: val_accuracy improved from -inf to 0.84680, saving model to ./cifar10_VGG19.h5
Epoch 2/20
196/196 [==============================] - 6s 32ms/step - loss: 0.3293 - accuracy: 0.8891 - val_loss: 0.4684 - val_accuracy: 0.8508

Epoch 00002: val_accuracy improved from 0.84680 to 0.85080, saving model to ./cifar10_VGG19.h5
Epoch 3/20
196/196 [==============================] - 6s 32ms/step - loss: 0.3078 - accuracy: 0.8966 - val_loss: 0.4579 - val_accuracy: 0.8596

Epoch 00003: val_accuracy improved from 0.85080 to 0.85960, saving model to ./cifar10_VGG19.h5
Epoch 4/20
196/196 [==============================] - 7s 35ms/step - loss: 0.2933 - accuracy: 0.9030 - val_loss: 0.4590 - val_accuracy: 0.8640

Epoch 00004: val_accuracy improved from 0.85960 to 0.86400, saving model to ./cifar10_VGG19.h5
Epoch 5/20
196/196 [===========================